In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os
from config import weather_api_key
from citipy import citipy
import requests
from xml.etree.ElementTree import tostring

In [6]:
# generate random world locations
lats = np.random.uniform(-90.000, 90.000, 2000)
lngs = np.random.uniform(-180.000, 180.000, 2000)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [9]:
cities = []

# Finds nearest city to coordinates
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)


len(cities)

726

In [19]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description
                          })

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

    # Indicate that Data Loading is complete.
    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | punta arenas
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 2 of Set 1 | saskylakh
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 3 of Set 1 | sol-iletsk
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 4 of Set 1 | zhanatas
City not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 5 of Set 1 | port hardy
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 1 | rikitea
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 7 of Set 1 | kulhudhuffushi
-----------------------------
Data Retrieval Complete      
------------------

In [20]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Punta Arenas,CL,-53.1500,-70.9167,42.91,81,0,14.97,clear sky
1,Saskylakh,RU,71.9167,114.0833,49.78,71,82,7.90,broken clouds
2,Sol-Iletsk,RU,51.1631,54.9918,80.35,38,18,2.80,few clouds
3,Port Hardy,CA,50.6996,-127.4199,60.80,83,55,2.68,broken clouds
4,Rikitea,PF,-23.1203,-134.9692,73.09,79,99,24.90,light rain
...,...,...,...,...,...,...,...,...,...
656,Kuybysheve,UA,47.3580,36.6491,90.12,19,85,10.45,overcast clouds
657,Puerto Colombia,CO,10.9878,-74.9547,82.49,72,100,5.41,overcast clouds
658,Santa Lucia,ES,27.9117,-15.5407,73.76,61,20,29.93,few clouds
659,Atikokan,CA,48.7555,-91.6216,70.27,43,2,10.51,clear sky


In [21]:
city_data_df.to_csv(os.path.join("WeatherPy_Database.csv"), index_label="City_ID")